In [1]:
!pip3 install --upgrade google-cloud-documentai
!pip3 install --upgrade google-cloud-storage

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 176 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 821 kB/s eta 0:00:01
  Using cached packaging-20.9-py2.py3-none-any.whl (40 kB)
  Using cached googleapis_common_protos-1.53.0-py2.py3-none-any.whl (198 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
     |████████████████████████████████| 1.0 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 142 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 2.9 MB/s eta 0:00:01
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached cachetools-4.2.1-py3-none-any.whl (12 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
You should consider upgrading via t

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from google.cloud import documentai_v1 as documentai
from google.cloud import storage

In [3]:
#!wget https://storage.googleapis.com/practical-ml-vision-book/images/scott_walker.pdf

In [2]:
!gcloud auth application-default login

/bin/sh: 1: gcloud: not found


In [5]:
!gcloud auth application-default set-quota-project edd-gcp-01

/tools/google-cloud-sdk/lib/third_party/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "edd-gcp-01" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [3]:
# TODO(developer): Uncomment these variables before running the sample.
project_id= 'edd-gcp-01'
location = 'eu' # Format is 'us' or 'eu'
processor_id = '10c5d397b0870d86' # Create processor in Cloud Console
file_path = 'EUR-MED_C1300_Skywalker.pdf'

In [4]:
def process_document_sample(
    project_id: str, location: str, processor_id: str, file_path: str
):
    from google.cloud import documentai_v1 as documentai

    # You must set the api_endpoint if you use a location other than 'us', e.g.:
    opts = {}
    if location == "eu":
        opts = {"api_endpoint": "eu-documentai.googleapis.com"}

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    with open(file_path, "rb") as image:
        image_content = image.read()

    # Read the file into memory
    document = {"content": image_content, "mime_type": "application/pdf"}

    # Configure the process request
    request = {"name": name, "raw_document": document}

    # Recognizes text entities in the PDF document
    result = client.process_document(request=request)

    document = result.document

    print("Document processing complete.")

    # For a full list of Document object attributes, please reference this page: https://googleapis.dev/python/documentai/latest/_modules/google/cloud/documentai_v1beta3/types/document.html#Document

    document_pages = document.pages

    # Read the text recognition output from the processor
    print("The document contains the following paragraphs:")
    for page in document_pages:
        paragraphs = page.paragraphs
        for paragraph in paragraphs:
            paragraph_text = get_text(paragraph.layout, document)
            print(f"Paragraph text: {paragraph_text}")


In [5]:
# Extract shards from the text field
def get_text(doc_element: dict, document: dict):
    """
    Document AI identifies form fields by their offsets
    in document text. This function converts offsets
    to text snippets.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in doc_element.text_anchor.text_segments:
        start_index = (
            int(segment.start_index)
            if segment in doc_element.text_anchor.text_segments
            else 0
        )
        end_index = int(segment.end_index)
        response += document.text[start_index:end_index]
    return response

In [6]:
process_document_sample(project_id, location, processor_id, file_path)

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started